## Imports and Configurations

In [30]:
import os.path
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pickle

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

from sklearn.utils import shuffle

%matplotlib inline

In [16]:
# Configurations here
data_folder = './data/udacity_data'
training_file = 'train.p'
training_image_size = (160, 320)

flags = {
    'create_training_pickle': False,
    'train_model': True,
    'show_plots': False
}

## Load and Preprocess Data

In [17]:
df = pd.read_csv(os.path.join(data_folder, 'driving_log.csv'))
df.head()

,center,left,right,steering,throttle,brake,speed
0,IMG/center_2016_12_01_13_30_48_287.jpg,IMG/left_2016_12_01_13_30_48_287.jpg,IMG/right_2016_12_01_13_30_48_287.jpg,0.0,0.0,0.0,22.148290
1,IMG/center_2016_12_01_13_30_48_404.jpg,IMG/left_2016_12_01_13_30_48_404.jpg,IMG/right_2016_12_01_13_30_48_404.jpg,0.0,0.0,0.0,21.879630
2,IMG/center_2016_12_01_13_31_12_937.jpg,IMG/left_2016_12_01_13_31_12_937.jpg,IMG/right_2016_12_01_13_31_12_937.jpg,0.0,0.0,0.0,1.453011
3,IMG/center_2016_12_01_13_31_13_037.jpg,IMG/left_2016_12_01_13_31_13_037.jpg,IMG/right_2016_12_01_13_31_13_037.jpg,0.0,0.0,0.0,1.438419
4,IMG/center_2016_12_01_13_31_13_177.jpg,IMG/left_2016_12_01_13_31_13_177.jpg,IMG/right_2016_12_01_13_31_13_177.jpg,0.0,0.0,0.0,1.418236


In [18]:
if flags['create_training_pickle']:
    pickle_data = {}
    images = []

    for index, row in df.iterrows():
        img = mpimg.imread(os.path.join(data_folder, row['center']))
        img = img[np.newaxis, ...]
        images.append(img)

    pickle_data['features'] = np.concatenate(images, axis=0)
    pickle_data['labels'] = df['steering'].values

    with open(os.path.join(data_folder, training_file), 'wb') as f:
        pickle.dump(pickle_data, f)

In [31]:
with open(os.path.join(data_folder, training_file), 'rb') as f:
    data = pickle.load(f)

X_train = data['features']
y_train = data['labels']

X_train, y_train = shuffle(X_train, y_train)

In [20]:
np.min(y_train), np.mean(y_train), np.median(y_train), np.max(y_train)

(-0.94269539999999996, 0.0040696440648332532, 0.0, 1.0)

#### Balance Dataset

In [21]:
if flags['show_plots']:
    plt.hist(y_train, bins=[-1, -.01, .01, 1])

In [22]:
straight_indexes = [index for index, angle in enumerate(y_train) if angle > -.01 and angle < .01]
left_indexes = [index for index, angle in enumerate(y_train) if angle  < -.01]
right_indexes = [index for index, angle in enumerate(y_train) if angle > .01]

In [23]:
keep_num = min(len(straight_indexes), len(left_indexes), len(right_indexes))
keep_straight_indexes = np.random.choice(straight_indexes, keep_num)
keep_left_indexes = np.random.choice(left_indexes, keep_num)
keep_right_indexes = np.random.choice(right_indexes, keep_num)

## Setup and Train Network

In [32]:
model = Sequential()


model.add(Convolution2D(48, 11, 11, subsample=(4,4), input_shape=(160, 320, 3))) # 38 x 78
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2))) # 18 x 38


model.add(Convolution2D(256, 5, 5)) # 16 x 36
model.add(Activation('relu'))
model.add(MaxPooling2D((2,2))) # 8 x 18

model.add(Convolution2D(384, 3, 3, border_mode='same')) # 8 x 18
model.add(Activation('relu'))

model.add(Convolution2D(384, 3, 3, border_mode='same')) # 8 x 18
model.add(Activation('relu'))

model.add(Convolution2D(128, 3, 3, border_mode='same')) # 8 x 18
model.add(Activation('relu'))
model.add(MaxPooling2D((2,3))) # 4 x 6

model.add(Flatten())
model.add(Dense(3072))
model.add(Dense(3072))
model.add(Dense(1))

In [33]:
if flags['train_model']:
    model.compile('adam', 'mse')
    model.fit(X_train, y_train, nb_epoch=8, batch_size=128, validation_split=.2)
    model.save('model.h5')

Train on 6428 samples, validate on 1608 samples
Epoch 1/8
6428/6428 [==============================] - 39s - loss: 69.5027 - val_loss: 0.0185
Epoch 2/8
6428/6428 [==============================] - 39s - loss: 0.0171 - val_loss: 0.0192
Epoch 3/8
6428/6428 [==============================] - 39s - loss: 0.0172 - val_loss: 0.0186
Epoch 4/8
6428/6428 [==============================] - 39s - loss: 0.0167 - val_loss: 0.0195
Epoch 5/8
6428/6428 [==============================] - 39s - loss: 0.0163 - val_loss: 0.0202
Epoch 6/8
6428/6428 [==============================] - 39s - loss: 0.0161 - val_loss: 0.0182
Epoch 7/8
6428/6428 [==============================] - 39s - loss: 0.0161 - val_loss: 0.0227
Epoch 8/8
6428/6428 [==============================] - 39s - loss: 0.0165 - val_loss: 0.0177
